In [15]:
import pandas as pd
import numpy as np

In [3]:
validate_train_url = "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/DZW9I4MwAJrl_A"
validate_test_url = "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/8nSFbNQY92HCng"
validate_answers_train_url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/UsSATGKzLrhBFQ'
validate_answers_test = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/iu8jIJk1C15mww'
history_train_url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/BkZWvVVDjfB1rw'
users_train_url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/SEz-05NG0vpkKA'
validate_train = pd.read_csv(validate_train_url, sep='\t')
validate_test = pd.read_csv(validate_test_url, sep='\t')
validate_answers_train = pd.read_csv(validate_answers_train_url, sep='\t')
validate_answers_test = pd.read_csv(validate_answers_test, sep='\t')
history_train = pd.read_csv(history_train_url, sep='\t')
users_train = pd.read_csv(users_train_url, sep='\t')

In [4]:
validate_train = validate_train.drop('Unnamed: 0', axis=1)
validate_test = validate_test.drop('Unnamed: 0', axis=1)
validate_answers_train = validate_answers_train.drop('Unnamed: 0', axis=1)
validate_answers_test = validate_answers_test.drop('Unnamed: 0', axis=1)
history_train = history_train.drop('Unnamed: 0', axis=1)
users_train = users_train.drop('Unnamed: 0', axis=1)

In [5]:
views_for_users_by_publishers = {}
for user_id in users_train['user_id'].unique():
  temp = history_train[history_train['user_id'] == user_id]
  publishers = temp['publisher'].tolist()
  views_for_users_by_publishers[user_id] = []
  for i in range(22):
    views_for_users_by_publishers[user_id].append(publishers.count(i))

In [6]:
users_statistics_by_publishers = {}
for user_id in list(users_train['user_id'].unique()):
    user_history = history_train[history_train['user_id'] == user_id]
    for i in range(1,22):
        temp_user_history = user_history[user_history['publisher'] == i]
        users_statistics_by_publishers[(user_id, i, 'total_views')] = temp_user_history.shape[0]
        users_statistics_by_publishers[(user_id, i, 'views_share')] = temp_user_history.shape[0]/user_history.shape[0] if user_history.shape[0] != 0 else 0
        users_statistics_by_publishers[(user_id, i, 'min_cpm')] = temp_user_history['cpm'].min()
        users_statistics_by_publishers[(user_id, i, 'max_cpm')] = temp_user_history['cpm'].max()
        users_statistics_by_publishers[(user_id, i, 'mean_cpm')] = temp_user_history['cpm'].mean()
        users_statistics_by_publishers[(user_id, i, 'median_cpm')] = temp_user_history['cpm'].median()
        users_statistics_by_publishers[(user_id, i, 'mean_views_per_day')] = temp_user_history.shape[0]/((temp_user_history['hour'].max()-temp_user_history['hour'].min())/24) if temp_user_history['hour'].max()-temp_user_history['hour'].min() != 0 else 0



In [7]:
def count_views_by_user(user, publishers, views_for_users_by_publishers):
    sum = 0
    for publisher in publishers:
        sum += views_for_users_by_publishers[user][publisher]
    return sum

In [8]:
def make_target_dataset(validate_train, validate_answers_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers):
    target = []
    for index in range(validate_train.shape[0]):
        validate_row = validate_train.iloc[index]
        validate_answer_row = validate_answers_train.iloc[index]
        user_ids = list(map(int, (validate_row['user_ids'].replace('(', '').replace(')', '').replace(',', ' ')).split()))
        publisher_ids = list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split()))
        array = []
        for user_id in user_ids:
            array.append([user_id, count_views_by_user(user_id, publisher_ids, views_for_users_by_publishers)])
        array.sort(key=lambda item: item[1], reverse=True)
        users = {}
        for i in range(len(array)):
            users[array[i][0]] = 0
        for i in range(int(validate_answer_row['at_least_one']*len(array))):
            users[array[i][0]] = 1
        for i in range(int(validate_answer_row['at_least_two']*len(array))):
            users[array[i][0]] = 2
        for i in range(int(validate_answer_row['at_least_three']*len(array))):
            users[array[i][0]] = 3
        for user_id in user_ids:
            target.append(users[user_id])
    return pd.DataFrame({'target': target})
            
        

In [9]:
def make_feature_dataset(validate_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers):
    users_id = []
    cpm = []
    number_of_views = []
    duration = []
    min_cpm = []
    max_cpm = []
    mean_cpm = []
    median_cpm = []
    mean_views_per_day = []
    number_of_publisher = []
    for index in range(validate_train.shape[0]):
        validate_row = validate_train.iloc[index]
        validate_answer_row = validate_answers_train.iloc[index]
        user_ids = list(map(int, (validate_row['user_ids'].replace('(', '').replace(')', '').replace(',', ' ')).split()))
        publisher_ids = list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split()))
        
        for user_id in user_ids:
            duration.append(validate_row['hour_end']-validate_row['hour_start'])
            users_id.append(user_id)
            cpm.append(validate_row['cpm'])
            number_of_views.append(count_views_by_user(user_id, publisher_ids, views_for_users_by_publishers))
            number_of_publisher.append(len(publisher_ids))
            temp_min = []
            temp_max = []
            temp_mean = []
            temp_median = []
            temp_mean_views = []
            for publisher_id in list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split())):
                temp_min.append(users_statistics_by_publishers[(user_id, publisher_id, 'min_cpm')])
                temp_max.append(users_statistics_by_publishers[(user_id, publisher_id, 'max_cpm')])
                temp_mean.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_cpm')])
                temp_median.append(users_statistics_by_publishers[(user_id, publisher_id, 'median_cpm')])
                temp_mean_views.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_views_per_day')])
            min_cpm.append(min(temp_min))
            max_cpm.append(max(temp_max))
            temp_median.sort()                                  
            middle = temp_median[len(temp_median) // 2]        
            median_cpm.append(middle)
            mean_cpm.append(sum(temp_mean)/len(temp_mean))
            mean_views_per_day.append(sum(temp_mean_views)/len(temp_mean_views))
    return pd.DataFrame({
        "user_id": users_id,
        "cpm": cpm,
        'number_of_views': number_of_views,
        'min_cpm_per_publishers': min_cpm,
        'max_cpm_per_publishers': max_cpm,
        'mean_cpm_per_publishers': mean_cpm,
        'median_cpm_per_publishers': median_cpm,
        'mean_views_per_day': mean_views_per_day,
        'durations': duration,
        'number_of_publisher': number_of_publisher
    })

In [10]:
import random
from sklearn.model_selection import train_test_split

random.seed(42)

def preparing_data(validate_train, validate_answers_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers):
    y = make_target_dataset(validate_train, validate_answers_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
    X = make_feature_dataset(validate_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
    data = pd.concat([X, y], axis=1)
    data = data.reindex(random.shuffle([i for i in range(data.shape[0])])).reset_index(drop=True)
    data.fillna(0, inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['target']), data['target'], test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [11]:
X_train, X_test, y_train, y_test = preparing_data(validate_train, validate_answers_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)

In [12]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [17]:
pipelines = {
    'logreg': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(multi_class='multinomial', solver='saga', max_iter=5000))
    ]),
    'rf': Pipeline([
        ('clf', RandomForestClassifier(random_state=42))
    ]),
    'xgb_gpu': Pipeline([
        ('clf', XGBClassifier(
            objective='multi:softprob',
            num_class=4,
            use_label_encoder=False,
            eval_metric='mlogloss',
            device='cuda',        
            tree_method='hist',   
            random_state=42
        ))
    ])
}

param_grids = {
    'logreg': {
        'clf__C': [0.01, 0.1, 1, 10],
        'clf__penalty': ['l2']
    },
    'rf': {
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [None, 10, 20],
        'clf__min_samples_split': [2, 5]
    },
    'xgb_gpu': {
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [3, 5],
        'clf__learning_rate': [0.01, 0.1],
    }
}

In [18]:
best_estimators = {}

for name in pipelines:
    print(f"\n===== GridSearch for {name} =====")
    grid = GridSearchCV(
        estimator=pipelines[name],
        param_grid=param_grids[name],
        scoring='accuracy',      # можно 'f1_macro', 'roc_auc_ovr' и т.д.
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)
    print(f"Best params for {name}: {grid.best_params_}")
    print(f"Best CV score: {grid.best_score_:.4f}")

    best_estimators[name] = grid.best_estimator_

for name, est in best_estimators.items():
    y_pred = est.predict(X_test)
    print(f"\n*** Report for {name} ***")
    print(classification_report(y_test, y_pred, digits=4))


===== GridSearch for logreg =====
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params for logreg: {'clf__C': 0.01, 'clf__penalty': 'l2'}
Best CV score: 0.9433

===== GridSearch for rf =====
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params for rf: {'clf__max_depth': None, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}
Best CV score: 0.9951

===== GridSearch for xgb_gpu =====
Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:25:40] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:25:40] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

Best params for xgb_gpu: {'clf__learning_rate': 0.1, 'clf__max_depth': 5, 'clf__n_estimators': 200}
Best CV score: 0.9954

*** Report for logreg ***
              precision    recall  f1-score   support

           0     0.9699    0.9924    0.9810     45838
           1     0.3630    0.3072    0.3327      1846
           2     0.5000    0.0014    0.0028       716
           3     0.7878    0.7357    0.7609       999

    accuracy                         0.9472     49399
   macro avg     0.6552    0.5092    0.5193     49399
weighted avg     0.9367    0.9472    0.9381     49399


*** Report for rf ***
              precision    recall  f1-score   support

           0     0.9981    0.9992    0.9987     45838
           1     0.9616    0.9496    0.9556      1846
           2     0.9638    0.9302    0.9467       716
           3     0.9840    0.9840    0.9840       999

    accuracy                         0.9960     49399
   macro avg     0.9769    0.9657    0.9712     49399
weighted avg 

In [28]:
def predict(model, validate_test):
    at_least_one = []
    at_least_two = []
    at_least_three = []
    for index in range(validate_test.shape[0]):
        row = validate_test.iloc[index]
        X_test = make_feature_dataset(pd.DataFrame([row]) , views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
        pred = model.predict(X_test)
        at_least_one.append(list(pred).count(1)/len(pred))
        at_least_two.append(list(pred).count(2)/len(pred))
        at_least_three.append(list(pred).count(3)/len(pred))
    return pd.DataFrame({
        "at_least_one": at_least_one,
        "at_least_two": at_least_two,
        'at_least_three': at_least_three
    })

In [21]:
def load_answers(answers_filename):
    return pd.read_csv(answers_filename, sep="\t")


def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )


In [37]:
pred = predict(best_estimators['xgb_gpu'], validate_train)

In [39]:
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')


Percentage Error on Test: 285.82%


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
